In [1]:
import pandas as pd
import seaborn as sbs
import numpy as np
import calendar
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [2]:
subte = pd.read_csv("data/estaciones-de-subte.csv")

In [3]:
properati = pd.read_csv("data/properati_full_month.csv",low_memory=False)

In [4]:
test = pd.read_csv("data/properati_dataset_testing_noprice.csv")

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [6]:
properati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640710 entries, 0 to 640709
Data columns (total 29 columns):
country_name                  263989 non-null object
created_on                    640710 non-null object
currency                      532703 non-null object
description                   428806 non-null object
expenses                      37716 non-null object
extra                         164828 non-null object
floor                         80294 non-null float64
geonames_id                   541207 non-null float64
id                            263989 non-null object
image_thumbnail               620269 non-null object
lat                           480309 non-null float64
lat-lon                       480309 non-null object
lon                           480304 non-null float64
operation                     640710 non-null object
place_name                    640658 non-null object
place_with_parent_names       640710 non-null object
price                         540315 no

# Dropeo de columnas, asi quedan misma cantidad de columnas en el train ( properati ) y en el test, solo me quedo con price_aprox_usd

In [7]:
properati.drop(inplace=True,labels=\
               ['image_thumbnail','title','surface_in_m2','price_aprox_local_currency','price',\
               'price_per_m2','price_usd_per_m2','properati_url','title','geonames_id','currency','extra'],axis=1)

In [8]:
test['country_name'].value_counts(dropna=False)

Argentina    14166
Name: country_name, dtype: int64

In [9]:
test['property_type'].value_counts(dropna=False)

departamento    9761
casa            3409
ph               996
Name: property_type, dtype: int64

In [10]:
test['state_name'].value_counts()

Capital Federal             8847
Bs.As. G.B.A. Zona Oeste    3101
Bs.As. G.B.A. Zona Sur      2215
Bs.As. G.B.A. Zona Norte       2
Buenos Aires Interior          1
Name: state_name, dtype: int64

In [11]:
test['place_name'].value_counts()

Palermo                       1020
Belgrano                       783
Caballito                      650
La Plata                       603
Recoleta                       556
Ituzaingó                      547
Ramos Mejía                    394
Morón                          393
Almagro                        376
Castelar                       345
Flores                         329
Villa Crespo                   329
Balvanera                      327
Villa Urquiza                  309
Nuñez                          300
Barrio Norte                   276
Capital Federal                247
Monserrat                      221
Saavedra                       211
San Telmo                      207
Lomas de Zamora                197
Quilmes                        178
Moreno                         171
Palermo Hollywood              164
Puerto Madero                  155
Banfield                       134
Haedo                          126
Villa Devoto                   123
Lanús               

# Parseo de datos

In [12]:
properati.country_name = properati.place_with_parent_names.apply(lambda x : x.split('|')[1])

In [13]:
properati = properati[properati['country_name'] == 'Argentina']

In [14]:
states = properati.place_with_parent_names.apply(lambda x : x.split('|')[2])

In [15]:
testStates = test.place_with_parent_names.apply(lambda x: x.split('|')[2])

In [16]:
properati.loc[:,'state_name'] = states

In [17]:
test.loc[:,'state_name'] = testStates

In [18]:
properati = properati[
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Norte' ) |
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Sur' ) |
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Oeste' ) |
    (properati['state_name'] == 'Capital Federal' ) 
    ]

In [19]:
properati['zone'] = properati.place_with_parent_names.apply(lambda x : x.split('|')[3])

In [20]:
test['zone'] = test.place_with_parent_names.apply(lambda x : x.split('|')[3])

In [21]:
properati['zone'].value_counts()

Tigre                    45521
                         24438
Palermo                  22482
Vicente López            21566
La Matanza               20195
Pilar                    18257
San Isidro               17795
Belgrano                 17506
Morón                    16671
La Plata                 14707
Caballito                11919
General San Martín        9673
Tres de Febrero           8901
Barrio Norte              8261
Villa Urquiza             7974
Villa Crespo              7923
Escobar                   7654
Ituzaingó                 7622
Lanús                     7540
Flores                    7447
Lomas de Zamora           7145
San Fernando              7077
Almagro                   6772
Recoleta                  6670
Nuñez                     6120
San Miguel                5723
Boedo                     4697
Balvanera                 4397
Quilmes                   4344
Almirante Brown           3814
                         ...  
Paternal                   971
Villa Lu

In [22]:
properati[properati['zone'] == 'Capital Federal']

,country_name,created_on,description,expenses,floor,id,lat,lat-lon,lon,operation,place_name,place_with_parent_names,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,zone


In [23]:
properati['place_name'].value_counts()

Capital Federal                       18865
Belgrano                              17506
Tigre                                 16740
Nordelta                              16011
Palermo                               14459
Pilar                                 14215
Caballito                             11919
La Plata                              11851
Morón                                  8312
Olivos                                 8312
Barrio Norte                           8261
Villa Urquiza                          7974
Villa Crespo                           7923
Flores                                 7447
Ramos Mejía                            7189
La Matanza                             7074
San Isidro                             7046
Almagro                                6772
Recoleta                               6670
Ituzaingó                              6265
Nuñez                                  6120
Vicente López                          6107
Benavidez                       

In [24]:
properati[properati['zone'] == '']['state_name']

0                  Capital Federal
466                Capital Federal
476                Capital Federal
484                Capital Federal
488                Capital Federal
495                Capital Federal
499                Capital Federal
504                Capital Federal
505                Capital Federal
509                Capital Federal
510                Capital Federal
513                Capital Federal
514                Capital Federal
515                Capital Federal
516                Capital Federal
517                Capital Federal
530                Capital Federal
536                Capital Federal
540                Capital Federal
548                Capital Federal
549                Capital Federal
552                Capital Federal
553                Capital Federal
555                Capital Federal
556                Capital Federal
557                Capital Federal
558                Capital Federal
559                Capital Federal
560                C

In [25]:
test['zone'].value_counts()

Palermo                 1324
Morón                    961
Belgrano                 783
La Plata                 693
La Matanza               676
Ituzaingó                666
Caballito                650
Recoleta                 556
Lomas de Zamora          438
Almagro                  376
Flores                   329
Villa Crespo             329
Balvanera                327
Tres de Febrero          319
Villa Urquiza            309
Nuñez                    300
Moreno                   296
Barrio Norte             276
Quilmes                  273
                         265
Lanús                    227
Monserrat                221
Saavedra                 211
San Telmo                207
Puerto Madero            155
Villa Devoto             123
Almirante Brown          118
Colegiales               113
Mataderos                106
San Cristobal            106
                        ... 
Las Cañitas               48
Centro / Microcentro      46
Boca                      44
Hurlingham    

In [26]:
test[test['zone'] == 'Capital Federal']

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone


In [27]:
test[test['zone'] == '']['state_name'].value_counts()

Capital Federal             247
Bs.As. G.B.A. Zona Oeste     13
Bs.As. G.B.A. Zona Sur        4
Buenos Aires Interior         1
Name: state_name, dtype: int64

In [28]:
test[test['place_name'] == 'Capital Federal']

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone
63,2264155,2017-08-01,casa,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,"-34.6365236219,-58.4294294221",-34.636524,-58.429429,246.0,211.0,NaN,NaN,NaN,Casa de 4 ambientes con dependencia de servici...,
70,2264183,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,62.0,59.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,
74,2264206,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,62.0,59.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,
75,2264212,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,62.0,59.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,
76,2264215,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,62.0,59.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,
77,2264218,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,62.0,59.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,
79,2264221,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,62.0,59.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,
80,2264227,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,94.0,68.0,NaN,NaN,NaN,Departamento piso en Venta de 3 ambientes en P...,
82,2264234,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,117.0,91.0,NaN,NaN,NaN,Departamento piso en Venta de 4 ambientes en P...,
88,2264265,2017-08-01,departamento,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,115.0,62.0,NaN,NaN,NaN,Departamentos Semipisos en Venta de 3 ambiente...,


In [29]:
test['place_name'].value_counts()

Palermo                       1020
Belgrano                       783
Caballito                      650
La Plata                       603
Recoleta                       556
Ituzaingó                      547
Ramos Mejía                    394
Morón                          393
Almagro                        376
Castelar                       345
Flores                         329
Villa Crespo                   329
Balvanera                      327
Villa Urquiza                  309
Nuñez                          300
Barrio Norte                   276
Capital Federal                247
Monserrat                      221
Saavedra                       211
San Telmo                      207
Lomas de Zamora                197
Quilmes                        178
Moreno                         171
Palermo Hollywood              164
Puerto Madero                  155
Banfield                       134
Haedo                          126
Villa Devoto                   123
Lanús               

# Relleno los que en zone no tienen nada, los relleno con el place_name

In [30]:
properati.loc[properati['zone'] == '','zone'] = properati.loc[properati['zone'] == '','place_name']

In [31]:
properati['zone'].value_counts()

Tigre                       45521
Palermo                     22482
Vicente López               21566
La Matanza                  20195
Capital Federal             18865
Pilar                       18257
San Isidro                  17795
Belgrano                    17506
Morón                       16671
La Plata                    14707
Caballito                   11919
General San Martín           9673
Tres de Febrero              8901
Barrio Norte                 8261
Villa Urquiza                7974
Villa Crespo                 7923
Escobar                      7654
Ituzaingó                    7622
Lanús                        7540
Flores                       7447
Lomas de Zamora              7145
San Fernando                 7077
Almagro                      6772
Recoleta                     6670
Nuñez                        6120
San Miguel                   5723
Bs.As. G.B.A. Zona Norte     4779
Boedo                        4697
Balvanera                    4397
Quilmes       

In [32]:
test.loc[test['zone'] == '','zone'] = test.loc[test['zone'] == '','place_name']

In [33]:
test['zone'].value_counts()

Palermo                     1324
Morón                        961
Belgrano                     783
La Plata                     693
La Matanza                   676
Ituzaingó                    666
Caballito                    650
Recoleta                     556
Lomas de Zamora              438
Almagro                      376
Villa Crespo                 329
Flores                       329
Balvanera                    327
Tres de Febrero              319
Villa Urquiza                309
Nuñez                        300
Moreno                       296
Barrio Norte                 276
Quilmes                      273
Capital Federal              247
Lanús                        227
Monserrat                    221
Saavedra                     211
San Telmo                    207
Puerto Madero                155
Villa Devoto                 123
Almirante Brown              118
Colegiales                   113
San Cristobal                106
Mataderos                    106
          

In [34]:
properati.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 446226 entries, 0 to 640708
Data columns (total 19 columns):
country_name               446226 non-null object
created_on                 446226 non-null object
description                299461 non-null object
expenses                   32722 non-null object
floor                      68329 non-null float64
id                         181269 non-null object
lat                        335779 non-null float64
lat-lon                    335779 non-null object
lon                        335774 non-null float64
operation                  446226 non-null object
place_name                 446174 non-null object
place_with_parent_names    446226 non-null object
price_aprox_usd            398410 non-null float64
property_type              446226 non-null object
rooms                      249377 non-null float64
state_name                 446226 non-null object
surface_covered_in_m2      166808 non-null float64
surface_total_in_m2        136153 n

In [35]:
properati.head()

,country_name,created_on,description,expenses,floor,id,lat,lat-lon,lon,operation,place_name,place_with_parent_names,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,zone
0,Argentina,2013-08-01,NaN,NaN,1.0,NaN,-34.598942,"-34.598942,-58.500647",-58.500647,sell,Capital Federal,|Argentina|Capital Federal|,130896.0,apartment,2.0,Capital Federal,NaN,NaN,Capital Federal
2,Argentina,2013-08-01,NaN,NaN,NaN,NaN,-34.591586,"-34.591586,-58.385966",-58.385966,sell,Recoleta,|Argentina|Capital Federal|Recoleta|,595000.0,apartment,2.0,Capital Federal,NaN,NaN,Recoleta
3,Argentina,2013-08-01,NaN,NaN,NaN,NaN,-34.587855,"-34.587855,-58.395342",-58.395342,sell,Recoleta,|Argentina|Capital Federal|Recoleta|,140000.0,apartment,2.0,Capital Federal,NaN,NaN,Recoleta
4,Argentina,2013-08-01,NaN,NaN,NaN,NaN,-34.597944,"-34.597944,-58.415853",-58.415853,sell,Palermo,|Argentina|Capital Federal|Palermo|,118000.0,apartment,2.0,Capital Federal,NaN,NaN,Palermo
5,Argentina,2013-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sell,Berazategui,|Argentina|Bs.As. G.B.A. Zona Sur|Berazategui|...,800000.0,house,5.0,Bs.As. G.B.A. Zona Sur,NaN,NaN,Berazategui


# Se usa zone en vez de place_name

In [36]:
properati['property_type'].value_counts()

apartment    268973
house        140237
PH            27310
store          9706
Name: property_type, dtype: int64

In [37]:
test['property_type'].value_counts()

departamento    9761
casa            3409
ph               996
Name: property_type, dtype: int64

In [38]:
# Filtro las stores
properati = properati.loc[properati['property_type'] != 'store',:]

In [39]:
properati['property_type'].value_counts()

apartment    268973
house        140237
PH            27310
Name: property_type, dtype: int64

In [40]:
properati['property_type'] = properati['property_type'].map({'apartment':0, 'house':1, 'PH' :2})

In [41]:
properati['property_type'].value_counts()

0    268973
1    140237
2     27310
Name: property_type, dtype: int64

In [42]:
test['property_type'] = test['property_type'].map({'departamento':0, 'casa':1, 'ph' :2})

In [43]:
test['property_type'].value_counts()

0    9761
1    3409
2     996
Name: property_type, dtype: int64

In [44]:
test['operation'].value_counts()

venta       14165
alquiler        1
Name: operation, dtype: int64

In [45]:
test['surface_covered_in_m2'].value_counts(dropna=False)

NaN           1161
 40.0          355
 100.0         311
 45.0          308
 60.0          280
 70.0          276
 80.0          241
 34.0          235
 50.0          235
 46.0          229
 42.0          218
 35.0          218
 30.0          207
 36.0          206
 39.0          197
 47.0          195
 32.0          190
 120.0         185
 44.0          185
 33.0          182
 38.0          176
 37.0          170
 43.0          170
 41.0          166
 90.0          158
 65.0          158
 55.0          158
 52.0          154
 48.0          154
 28.0          143
              ... 
 428.0           1
 435.0           1
 438.0           1
 439.0           1
 457.0           1
 359.0           1
 465.0           1
 466.0           1
 472.0           1
 479.0           1
 485.0           1
 487.0           1
 409.0           1
 408.0           1
 404.0           1
 399.0           1
 397.0           1
 396.0           1
 394.0           1
 388.0           1
 384.0           1
 383.0      

In [46]:
test[test['surface_covered_in_m2'] == 3243242.0 ]

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone
8492,2309655,2017-08-18,1,venta,Quilmes,|Argentina|Bs.As. G.B.A. Zona Sur|Quilmes|Quil...,Argentina,Bs.As. G.B.A. Zona Sur,"-34.718767,-58.24869",-34.718767,-58.24869,NaN,3243242.0,NaN,5.0,NaN,Corredor Responsable: Martin Veltri - C.M.Q 80...,Quilmes


In [47]:
properati['surface_covered_in_m2'].value_counts(dropna=False)

NaN          274863
 40.0          3905
 50.0          3556
 60.0          2935
 45.0          2935
 70.0          2708
 100.0         2700
 30.0          2625
 35.0          2448
 200.0         2440
 80.0          2388
 42.0          2370
 120.0         2182
 55.0          2159
 90.0          2114
 38.0          2097
 0.0           2083
 20.0          2063
 48.0          2009
 150.0         1893
 65.0          1799
 36.0          1797
 75.0          1787
 43.0          1775
 33.0          1754
 46.0          1750
 47.0          1749
 34.0          1702
 44.0          1672
 180.0         1658
              ...  
 1231.0           1
 749.0            1
 1236.0           1
 1240.0           1
 608.0            1
 607.0            1
 1112.0           1
 628.0            1
 934.0            1
 1092.0           1
 937.0            1
 940.0            1
 941.0            1
 652.0            1
 647.0            1
 952.0            1
 644.0            1
 965.0            1
 638.0            1


In [48]:
properati[properati['surface_covered_in_m2'] == 987897.0 ]

,country_name,created_on,description,expenses,floor,id,lat,lat-lon,lon,operation,place_name,place_with_parent_names,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,zone
462644,Argentina,2016-11-13,- Publicado a traves de VisualGestion -,NaN,NaN,4456c9c9d70a1a958af58f1ad87f027358b24ef0,NaN,NaN,NaN,sell,Morón,|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|Morón|,62000.0,0,2.0,Bs.As. G.B.A. Zona Oeste,987897.0,NaN,Morón


In [49]:
train = properati.copy()

# revisar si esto funciona o cambia a cualquier cosa

In [50]:
s = pd.Series(['a','b','c','e','d'])
a = pd.Series(['e','b','a'])

In [51]:
labelS,levelS = pd.factorize(s)

In [52]:
labelA,levelA = pd.factorize(a)

In [53]:
labelS

array([0, 1, 2, 3, 4])

In [54]:
labelA

array([0, 1, 2])

In [55]:
train['zona'] = train['zone'].rank(method='dense', ascending=False).astype(int)
test['zona'] = test['zone'].rank(method='dense', ascending=False).astype(int)
test['state'] = test['state_name'].map({'Capital Federal':0, 'Bs.As. G.B.A. Zona Norte':1, 'Bs.As. G.B.A. Zona Oeste' :2,\
                                       'Bs.As. G.B.A. Zona Sur': 3,'Buenos Aires Interior': 4})
train['state'] = train['state_name'].map({'Capital Federal':0, 'Bs.As. G.B.A. Zona Norte':1, 'Bs.As. G.B.A. Zona Oeste' :2,\
                                       'Bs.As. G.B.A. Zona Sur': 3})

In [56]:
train['state_name'].value_counts()

Capital Federal             191489
Bs.As. G.B.A. Zona Norte    138873
Bs.As. G.B.A. Zona Oeste     60523
Bs.As. G.B.A. Zona Sur       45635
Name: state_name, dtype: int64

In [57]:
train['state'].value_counts()

0    191489
1    138873
2     60523
3     45635
Name: state, dtype: int64

In [58]:
test['state_name'].value_counts()

Capital Federal             8847
Bs.As. G.B.A. Zona Oeste    3101
Bs.As. G.B.A. Zona Sur      2215
Bs.As. G.B.A. Zona Norte       2
Buenos Aires Interior          1
Name: state_name, dtype: int64

In [59]:
test['state'].value_counts()

0    8847
2    3101
3    2215
1       2
4       1
Name: state, dtype: int64

In [60]:
test['zone'].value_counts()

Palermo                     1324
Morón                        961
Belgrano                     783
La Plata                     693
La Matanza                   676
Ituzaingó                    666
Caballito                    650
Recoleta                     556
Lomas de Zamora              438
Almagro                      376
Villa Crespo                 329
Flores                       329
Balvanera                    327
Tres de Febrero              319
Villa Urquiza                309
Nuñez                        300
Moreno                       296
Barrio Norte                 276
Quilmes                      273
Capital Federal              247
Lanús                        227
Monserrat                    221
Saavedra                     211
San Telmo                    207
Puerto Madero                155
Villa Devoto                 123
Almirante Brown              118
Colegiales                   113
San Cristobal                106
Mataderos                    106
          

In [61]:
train['zone'].value_counts()

Tigre                       45284
Palermo                     21919
Vicente López               21178
La Matanza                  19727
Capital Federal             18556
Pilar                       18013
San Isidro                  17295
Belgrano                    17242
Morón                       16391
La Plata                    14252
Caballito                   11785
General San Martín           9401
Tres de Febrero              8714
Barrio Norte                 8020
Villa Urquiza                7879
Villa Crespo                 7634
Escobar                      7538
Ituzaingó                    7498
Flores                       7291
Lanús                        7258
San Fernando                 6955
Lomas de Zamora              6784
Almagro                      6655
Recoleta                     6491
Nuñez                        6035
San Miguel                   5623
Bs.As. G.B.A. Zona Norte     4680
Boedo                        4627
Quilmes                      4259
Balvanera     

# Las 3 columnas por ahora serian 'zone' 'state' y 'property_type'

### Saco la cercania a subtes

In [62]:
def subtes(x):
    # Veo si esta null o no
    if ( (x.lat) and (x.lon) ):
        # si esta en capital
        if ( x.state_name == 'Capital Federal'):
            # defino el rango a buscar
            rango = 0.0044
            latitud = x.lat
            longitud = x.lon
            coinciden = subte.loc[ (subte['Y'].between(latitud - rango, latitud + rango,inclusive=True)) & 
                 (subte['X'].between(longitud - rango, longitud + rango, inclusive=True) ),:]
            # si hay una linea cerca la devuelve
            if ( not(coinciden.empty) ):
                return coinciden['LINEA'].iloc[0]
            else:
                return 'N'
    # si no encuentra ninguno de los otros casos sale con N         
    return 'NULL'

In [63]:
test2 = test.copy()

In [64]:
resultado = test2.apply(subtes,axis=1)

In [65]:
resultado.value_counts()

N       5485
NULL    5319
D       1081
A        735
B        603
H        411
C        314
E        218
dtype: int64

In [66]:
test2['subte'] = resultado

In [67]:
subtes = train.apply(subtes,axis=1)

In [68]:
subtes.value_counts()

NULL    245031
N       121989
D        24573
B        14635
A        11446
H         7240
E         6706
C         4900
dtype: int64

In [69]:
train['subte'] = subtes

In [70]:
ewl = None
la = None
if not ewl and not la :
    print 'ew'

ew


In [71]:
# ds

In [72]:
test2

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,...,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone,zona,state,subte
0,3632,2017-08-24,0,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,...,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,Puerto Madero,27,0,N
1,3633,2017-08-25,0,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...,Buenos Aires Interior,70,4,NULL
2,2263404,2017-08-01,0,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,...,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...,Palermo,36,0,D
3,2263405,2017-08-01,0,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,...,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,Chacarita,65,0,N
4,2263406,2017-08-01,0,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,...,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...,Chacarita,65,0,N
5,2263407,2017-08-01,0,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,...,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,Chacarita,65,0,N
6,2263408,2017-08-01,0,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,...,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,Chacarita,65,0,N
7,2263409,2017-08-01,0,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,...,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,Chacarita,65,0,N
8,2263410,2017-08-01,0,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,...,67.0,67.0,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...,Colegiales,63,0,N
9,2263411,2017-08-01,0,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,NaN,...,47.0,47.0,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...,Almagro,82,0,N


In [73]:
test2[test2['state_name'] != 'Capital Federal']['subte'].value_counts()

NULL    5319
Name: subte, dtype: int64

In [74]:
test2['subte'] = test2['subte'].map({'NULL':0, 'N':1, 'A' :2, 'B' : 3, 'C': 4, 'D' : 5, 'E' : 6, 'H': 7})
train['subte'] = train['subte'].map({'NULL':0, 'N':1, 'A' :2, 'B' : 3, 'C': 4, 'D' : 5, 'E' : 6, 'H': 7})

In [75]:
train = train.loc[pd.notnull(train['price_aprox_usd'])]

In [76]:
trainToTest = train[['property_type','zona','state','subte']]

In [77]:
trainToTest

,property_type,zona,state,subte
0,0,81,0,1
2,0,30,0,1
3,0,30,0,7
4,0,42,0,1
5,1,88,3,0
6,0,27,0,7
7,0,30,0,1
8,1,35,1,0
10,0,90,0,4
11,0,27,0,7


In [78]:
trainToTest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390061 entries, 0 to 640708
Data columns (total 4 columns):
property_type    390061 non-null int64
zona             390061 non-null int32
state            390061 non-null int64
subte            390061 non-null int64
dtypes: int32(1), int64(3)
memory usage: 13.4 MB


In [79]:
train_price = train[['price_aprox_usd']]

In [80]:
train_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390061 entries, 0 to 640708
Data columns (total 1 columns):
price_aprox_usd    390061 non-null float64
dtypes: float64(1)
memory usage: 6.0 MB


In [81]:

x_train, x_test, y_train, y_test = train_test_split(trainToTest, train_price, test_size = 0.4, random_state = 0)

reg = linear_model.LinearRegression()

reg.fit(x_train, y_train) #entreno el algoritmo

#prediccion

pred = reg.predict(x_test)

score = sqrt(mean_squared_error(pred,y_test))
score

295799.7411391827

In [82]:
x = trainToTest.values

In [83]:
x

array([[ 0, 81,  0,  1],
       [ 0, 30,  0,  1],
       [ 0, 30,  0,  7],
       ..., 
       [ 1, 89,  0,  1],
       [ 1, 25,  1,  0],
       [ 0, 10,  0,  1]], dtype=int64)

In [84]:
from sklearn import preprocessing

x = trainToTest.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
trainToTest2 = pd.DataFrame(x_scaled)

D:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [85]:

x_train, x_test, y_train, y_test = train_test_split(trainToTest2, train_price, test_size = 0.4, random_state = 0)

reg = linear_model.LinearRegression(normalize=True)

reg.fit(x_train, y_train) #entreno el algoritmo

#prediccion

pred = reg.predict(x_test)

score = sqrt(mean_squared_error(pred,y_test))
score

295799.7411391827

In [86]:
trainToTest2

,0,1,2,3
0,0.0,0.833333,0.000000,0.142857
1,0.0,0.302083,0.000000,0.142857
2,0.0,0.302083,0.000000,1.000000
3,0.0,0.427083,0.000000,0.142857
4,0.5,0.906250,1.000000,0.000000
5,0.0,0.270833,0.000000,1.000000
6,0.0,0.302083,0.000000,0.142857
7,0.5,0.354167,0.333333,0.000000
8,0.0,0.927083,0.000000,0.571429
9,0.0,0.270833,0.000000,1.000000


In [87]:
train

,country_name,created_on,description,expenses,floor,id,lat,lat-lon,lon,operation,...,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,zone,zona,state,subte
0,Argentina,2013-08-01,NaN,NaN,1.0,NaN,-34.598942,"-34.598942,-58.500647",-58.500647,sell,...,130896.0,0,2.0,Capital Federal,NaN,NaN,Capital Federal,81,0,1
2,Argentina,2013-08-01,NaN,NaN,NaN,NaN,-34.591586,"-34.591586,-58.385966",-58.385966,sell,...,595000.0,0,2.0,Capital Federal,NaN,NaN,Recoleta,30,0,1
3,Argentina,2013-08-01,NaN,NaN,NaN,NaN,-34.587855,"-34.587855,-58.395342",-58.395342,sell,...,140000.0,0,2.0,Capital Federal,NaN,NaN,Recoleta,30,0,7
4,Argentina,2013-08-01,NaN,NaN,NaN,NaN,-34.597944,"-34.597944,-58.415853",-58.415853,sell,...,118000.0,0,2.0,Capital Federal,NaN,NaN,Palermo,42,0,1
5,Argentina,2013-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sell,...,800000.0,1,5.0,Bs.As. G.B.A. Zona Sur,NaN,NaN,Berazategui,88,3,0
6,Argentina,2013-08-02,NaN,NaN,2.0,NaN,-34.618726,"-34.618726,-58.403444",-58.403444,sell,...,70300.0,0,1.0,Capital Federal,NaN,NaN,San Cristobal,27,0,7
7,Argentina,2013-08-02,NaN,NaN,NaN,NaN,-34.590938,"-34.590938,-58.391122",-58.391122,sell,...,600000.0,0,3.0,Capital Federal,NaN,NaN,Recoleta,30,0,1
8,Argentina,2013-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sell,...,1100000.0,1,5.0,Bs.As. G.B.A. Zona Norte,NaN,NaN,Pilar,35,1,0
10,Argentina,2013-08-02,NaN,NaN,NaN,NaN,-34.592961,"-34.592961,-58.376981",-58.376981,sell,...,445000.0,0,2.0,Capital Federal,NaN,NaN,Barrio Norte,90,0,4
11,Argentina,2013-08-02,NaN,NaN,13.0,NaN,-34.618726,"-34.618726,-58.403444",-58.403444,sell,...,80800.0,0,1.0,Capital Federal,NaN,NaN,San Cristobal,27,0,7


In [95]:
trainToTest3 = train[['property_type','zone','state','subte']]

In [96]:
trainToTest3

,property_type,zone,state,subte
0,0,Capital Federal,0,1
2,0,Recoleta,0,1
3,0,Recoleta,0,7
4,0,Palermo,0,1
5,1,Berazategui,3,0
6,0,San Cristobal,0,7
7,0,Recoleta,0,1
8,1,Pilar,1,0
10,0,Barrio Norte,0,4
11,0,San Cristobal,0,7


In [103]:
trainToTest3.shape

(390061, 4)

In [89]:
# one_hot_encoding = pd.get_dummies(trainToTest3['zone'])
# trainToTest3 = trainToTest3.drop('zone',axis=1)
# trainToTest3 = trainToTest3.join(one_hot_encoding)
# trainToTest3

In [90]:

# x_train, x_test, y_train, y_test = train_test_split(trainToTest3, train_price, test_size = 0.4, random_state = 0)

# reg = linear_model.LinearRegression(normalize=True)

# reg.fit(x_train, y_train) #entreno el algoritmo

# #prediccion

# pred = reg.predict(x_test)

# score = sqrt(mean_squared_error(pred,y_test))
# score

In [97]:
test3 = test2.copy()

In [98]:
test3 = test3[['property_type','zone','state','subte']]

In [99]:
test3

,property_type,zone,state,subte
0,0,Puerto Madero,0,1
1,0,Buenos Aires Interior,4,0
2,0,Palermo,0,5
3,0,Chacarita,0,1
4,0,Chacarita,0,1
5,0,Chacarita,0,1
6,0,Chacarita,0,1
7,0,Chacarita,0,1
8,0,Colegiales,0,1
9,0,Almagro,0,1


In [100]:
zonas = test3[['zone']].copy()

In [101]:
zonas

,zone
0,Puerto Madero
1,Buenos Aires Interior
2,Palermo
3,Chacarita
4,Chacarita
5,Chacarita
6,Chacarita
7,Chacarita
8,Colegiales
9,Almagro


In [114]:
zonaDeTest = zonas['zone'].value_counts().keys()
zonaDeTest

Index([u'Palermo', u'Morón', u'Belgrano', u'La Plata', u'La Matanza',
       u'Ituzaingó', u'Caballito', u'Recoleta', u'Lomas de Zamora', u'Almagro',
       u'Villa Crespo', u'Flores', u'Balvanera', u'Tres de Febrero',
       u'Villa Urquiza', u'Nuñez', u'Moreno', u'Barrio Norte', u'Quilmes',
       u'Capital Federal', u'Lanús', u'Monserrat', u'Saavedra', u'San Telmo',
       u'Puerto Madero', u'Villa Devoto', u'Almirante Brown', u'Colegiales',
       u'San Cristobal', u'Mataderos', u'Avellaneda', u'Berazategui',
       u'Esteban Echeverría', u'Chacarita', u'Villa del Parque', u'Barracas',
       u'Villa Ortuzar', u'Boedo', u'Congreso', u'Merlo', u'Retiro',
       u'Floresta', u'San Nicolás', u'Liniers', u'Ezeiza', u'Parque Patricios',
       u'Parque Chacabuco', u'Coghlan', u'Villa Luro', u'Villa Pueyrredón',
       u'Once', u'Las Cañitas', u'Centro / Microcentro', u'Boca',
       u'Hurlingham', u'Paternal', u'Villa Lugano', u'Abasto',
       u'General Rodríguez', u'Constitución', u'M

In [115]:
zonaDeTrain = trainToTest3['zone'].value_counts().keys()
zonaDeTrain

Index([u'Tigre', u'Vicente López', u'Palermo', u'Capital Federal', u'Pilar',
       u'San Isidro', u'Belgrano', u'La Matanza', u'Morón', u'Caballito',
       u'La Plata', u'General San Martín', u'Barrio Norte', u'Villa Urquiza',
       u'Escobar', u'Villa Crespo', u'Tres de Febrero', u'Lanús',
       u'San Fernando', u'Lomas de Zamora', u'Flores', u'Almagro', u'Recoleta',
       u'Nuñez', u'Ituzaingó', u'San Miguel', u'Bs.As. G.B.A. Zona Norte',
       u'Boedo', u'Balvanera', u'Almirante Brown', u'San Telmo',
       u'San Cristobal', u'Saavedra', u'Colegiales', u'Barracas', u'Quilmes',
       u'Puerto Madero', u'Villa Devoto', u'Floresta', u'Moreno',
       u'Villa del Parque', u'Malvinas Argentinas', u'Mataderos', u'Monserrat',
       u'Villa Luro', u'Avellaneda', u'Congreso', u'Villa Pueyrredón',
       u'Merlo', u'Liniers', u'Esteban Echeverría', u'Coghlan', u'Berazategui',
       u'Parque Patricios', u'Ezeiza', u'Parque Chacabuco', u'Once',
       u'Las Cañitas', u'Centro / Microce

# Esas son las que no estan en el test

In [120]:
(zonaDeTrain.difference(zonaDeTest)).value_counts()

Malvinas Argentinas         1
Catalinas                   1
San Isidro                  1
Bs.As. G.B.A. Zona Norte    1
Distrito de las Artes       1
Distrito Audiovisual        1
José C Paz                  1
San Fernando                1
Tigre                       1
Zona Portuaria              1
Pilar                       1
Escobar                     1
Distrito Tecnológico        1
San Miguel                  1
dtype: int64

# Las que debo quitar del test

In [132]:
(zonaDeTest.difference(zonaDeTrain)).value_counts()

Buenos Aires Interior    1
dtype: int64

In [121]:
test[test['zone'] == "Tigre"]

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone,zona,state


In [122]:
test[test['place_name'] == "Tigre"]

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone,zona,state


In [123]:
test[test['state_name'] == "Tigre"]

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,zone,zona,state


In [127]:
one_hot_encoding = pd.get_dummies(trainToTest3['zone'])
trainToTest3 = trainToTest3.drop('zone',axis=1)
trainToTest3 = trainToTest3.join(one_hot_encoding)
trainToTest3

,property_type,state,subte,Abasto,Agronomía,Almagro,Almirante Brown,Avellaneda,Balvanera,Barracas,...,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque,Zona Portuaria
0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
trainToTest4 = trainToTest3.drop(['Malvinas Argentinas','Catalinas','San Isidro','Bs.As. G.B.A. Zona Norte', \
                                 'Distrito de las Artes','Distrito Audiovisual','José C Paz','San Fernando',\
                                 'Tigre','Zona Portuaria','Pilar','Escobar','Distrito Tecnológico','San Miguel'],axis=1)

In [130]:
trainToTest4.shape

(390061, 86)

In [133]:
one_hot_encoding = pd.get_dummies(test3['zone'])
test3 = test3.drop('zone',axis=1)
test3 = test3.join(one_hot_encoding)
test3

,property_type,state,subte,Abasto,Agronomía,Almagro,Almirante Brown,Avellaneda,Balvanera,Barracas,...,Villa Lugano,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque
0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
test3.shape

(14166, 87)

In [135]:
test3 = test3.drop(['Buenos Aires Interior'],axis=1)

In [136]:
test3.shape

(14166, 86)

# Revisar que tengan las mismas dimensiones

In [137]:

#x_train, x_test, y_train, y_test = train_test_split(trainToTest4, train_price, test_size = 0.4, random_state = 0)

reg = linear_model.LinearRegression(normalize=True)

reg.fit(trainToTest4, train_price) #entreno el algoritmo

#prediccion

pred = reg.predict(test3)

#score = sqrt(mean_squared_error(pred,y_test))
#score

In [138]:
pred

array([[ 798082.82879683],
       [ 398108.52532762],
       [ 257739.19024977],
       ..., 
       [ 209189.49547594],
       [ 310334.03731656],
       [ 169366.05494067]])

In [139]:
test['price_usd'] = pred

In [140]:
testASubir = test[['id','price_usd']]

In [141]:
testASubir.to_csv('data/SubmitOHESubtesLR.csv',index=False)